In [25]:
tweet = "I can't believe the battery on this phone is still at 90% after a full day!"
prompt = f"""Classify the sentiment of the following tweet:
Tweet: "{tweet}"
Sentiment (Positive, Negative, Neutral):"""


In [26]:
from openai import OpenAI

client = OpenAI(api_key="your_openai_api_key_here")

In [27]:
pip install openai tweepy pandas


In [18]:
import tweepy

client = tweepy.Client(
    bearer_token="YOUR_BEARER_TOKEN",
    consumer_key="YOUR_API_KEY",
    consumer_secret="YOUR_API_SECRET",
    access_token="YOUR_ACCESS_TOKEN",
    access_token_secret="YOUR_ACCESS_TOKEN_SECRET"
)

In [28]:
import tweepy
import re

class DataFetcherTool:
    def _init_(self, twitter_config):
        auth = tweepy.OAuthHandler(twitter_config['consumer_key'], twitter_config['consumer_secret'])
        auth.set_access_token(twitter_config['access_token'], twitter_config['access_token_secret'])
        self.api = tweepy.API(auth)

    def run(self, query, count=10):
        tweets = tweepy.Cursor(self.api.search_tweets,
                               q=query,
                               lang="en",
                               tweet_mode='extended').items(count)
        return [tweet.full_text for tweet in tweets]

class TextCleanerTool:
    def run(self, text):
        text = re.sub(r"http\S+|www\S+|https\S+", '', text)
        text = re.sub(r"@\w+|#\w+", '', text)
        text = re.sub(r"[^\w\s]", '', text)
        text = re.sub(r"\d+", '', text)
        return text.lower().strip()

In [29]:
import openai
import tweepy
import pandas as pd
import re

# ===============================
# CONFIGURATION
# ===============================
openai.api_key = "YOUR_OPENAI_API_KEY"

TWITTER_CONFIG = {
    "consumer_key": "YOUR_TWITTER_CONSUMER_KEY",
    "consumer_secret": "YOUR_TWITTER_CONSUMER_SECRET",
    "access_token": "YOUR_TWITTER_ACCESS_TOKEN",
    "access_token_secret": "YOUR_TWITTER_ACCESS_TOKEN_SECRET"
}

# ===============================
# TOOL: DataFetcherTool
# ===============================
class DataFetcherTool:
    def __init__(self, twitter_config):
        auth = tweepy.OAuthHandler(twitter_config["consumer_key"], twitter_config["consumer_secret"])
        auth.set_access_token(twitter_config["access_token"], twitter_config["access_token_secret"])
        self.api = tweepy.API(auth, wait_on_rate_limit=True)

    def run(self, query, count=10):
        tweets = []
        try:
            for tweet in tweepy.Cursor(self.api.search_tweets, q=query, lang="en", tweet_mode='extended').items(count):
                # Check for retweet and extract full_text accordingly
                text = tweet.full_text if not hasattr(tweet, 'retweeted_status') else tweet.retweeted_status.full_text
                tweets.append(text)
        except Exception as e:
            print(f"Error fetching tweets: {e}")
        return tweets

# ===============================
# TOOL: TextCleanerTool
# ===============================
class TextCleanerTool:
    def run(self, text):
        text = re.sub(r"http\S+|www\S+|https\S+", '', text)
        text = re.sub(r"@\w+|#\w+", '', text)
        text = re.sub(r"[^\w\s]", '', text)
        text = re.sub(r"\d+", '', text)
        return text.lower().strip()

# ===============================
# TOOL: SentimentAnalyzerTool (GPT)
# ===============================
class SentimentAnalyzerTool:
    def run(self, text):
        try:
            prompt = f"Classify the sentiment of the following tweet as Positive, Negative, or Neutral:\n\n\"{text}\"\n\nSentiment:"
            response = openai.Completion.create(
                engine="text-davinci-003",
                prompt=prompt,
                max_tokens=5,
                temperature=0
            )
            return response.choices[0].text.strip()
        except Exception as e:
            print(f"Error analyzing sentiment: {e}")
            return "Unknown"

# ===============================
# AGENT
# ===============================
class TwitterSentimentAgent:
    def __init__(self, tools):
        self.fetcher = tools['fetcher']
        self.cleaner = tools['cleaner']
        self.analyzer = tools['analyzer']

    def analyze(self, query, count=10):
        print(f"Fetching tweets for query: {query}")
        tweets = self.fetcher.run(query, count)
        results = []
        print(f"Analyzing {len(tweets)} tweets...")
        for tweet in tweets:
            cleaned = self.cleaner.run(tweet)
            sentiment = self.analyzer.run(cleaned)
            results.append({"original": tweet, "cleaned": cleaned, "sentiment": sentiment})
        return pd.DataFrame(results)

# ===============================
# MAIN
# ===============================
def main():
    tools = {
        "fetcher": DataFetcherTool(TWITTER_CONFIG),
        "cleaner": TextCleanerTool(),
        "analyzer": SentimentAnalyzerTool()
    }

    agent = TwitterSentimentAgent(tools)
    df = agent.analyze("OpenAI", count=5)
    print(df)
    df.to_csv("sentiment_results.csv", index=False)

if __name__ == "__main__":
    main()


Fetching tweets for query: OpenAI
Error fetching tweets: 401 Unauthorized
89 - Invalid or expired token.
Analyzing 0 tweets...
Empty DataFrame
Columns: []
Index: []
